In [19]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.tree import DecisionTreeClassifier



In [2]:
df=pd.read_csv('Loan_data.csv')
df.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N


In [3]:
df.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [4]:
df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].mean())
df['Credit_History'] = df['Credit_History'].fillna(df['Credit_History'].median())

In [5]:
df.dropna(inplace=True)
df.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [6]:
df['Loan_Status'].replace('Y',1,inplace=True)
df['Loan_Status'].replace('N',0,inplace=True)

In [7]:
df.Gender=df.Gender.map({'Male':1,'Female':0})
df['Gender'].value_counts()

1    444
0     98
Name: Gender, dtype: int64

In [8]:
df.Married=df.Married.map({'Yes':1,'No':0})
df['Married'].value_counts()

1    355
0    187
Name: Married, dtype: int64

In [9]:
df.Dependents=df.Dependents.map({'0':0,'1':1,'2':2,'3+':3})
df['Dependents'].value_counts()

0    309
1     94
2     94
3     45
Name: Dependents, dtype: int64

In [10]:
df.Education=df.Education.map({'Graduate':1,'Not Graduate':0})
df['Education'].value_counts()

1    425
0    117
Name: Education, dtype: int64

In [11]:
df.Self_Employed=df.Self_Employed.map({'Yes':1,'No':0})
df['Self_Employed'].value_counts()

0    467
1     75
Name: Self_Employed, dtype: int64

In [12]:
df.Property_Area=df.Property_Area.map({'Urban':2,'Rural':0,'Semiurban':1})
df['Property_Area'].value_counts()

1    209
2    174
0    159
Name: Property_Area, dtype: int64

In [13]:
df['Credit_History'].value_counts()

1.0    468
0.0     74
Name: Credit_History, dtype: int64

In [14]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,1,0,5849,0.0,146.412162,360.0,1.0,2,1
1,LP001003,1,1,1,1,0,4583,1508.0,128.000000,360.0,1.0,0,0
2,LP001005,1,1,0,1,1,3000,0.0,66.000000,360.0,1.0,2,1
3,LP001006,1,1,0,0,0,2583,2358.0,120.000000,360.0,1.0,2,1
4,LP001008,1,0,0,1,0,6000,0.0,141.000000,360.0,1.0,2,1


In [20]:
df.shape

(542, 13)

### Making Model


In [15]:
x=df.iloc[:, 1:12].values
y=df['Loan_Status']

In [21]:
model=DecisionTreeClassifier()
score=cross_val_score(model,x,y,cv=5,scoring='accuracy')
score.mean()

0.7343017329255861

### tuning model

In [ ]:
# prem={
#     'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
#     'splitter' :['best','random'],
#     'max_depth':[10,20,30,60,80,100,150,200],
#     'min_samples_split' : [20,30,40,60,80,100],
#     'min_samples_leaf': [20,30,40,60,80,100],
#     'max_features' : [2,4,6,8,9],
#     'max_leaf_nodes':[20,30,40,60,80,100],
#     'min_impurity_decrease' :[0.0,0.01,0.03,0.04,0.06,0.08,1]
# }

In [85]:
# prem={
#     'min_impurity_decrease' :[0.0,0.01,0.03,0.04,0.06,0.08,1]
# }

In [86]:
model=DecisionTreeClassifier(min_samples_split=150,max_depth=30,min_samples_leaf= 20,max_features=10,max_leaf_nodes=20)

In [87]:
gcv=GridSearchCV(model,param_grid=prem,cv=5,scoring='accuracy',n_jobs=-1)
gcv.fit(x,y)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=30, max_features=10,
                                              max_leaf_nodes=20,
                                              min_samples_leaf=20,
                                              min_samples_split=150),
             n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0, 0.01, 0.03, 0.04, 0.06,
                                                   0.08, 1]},
             scoring='accuracy')

In [88]:
gcv.best_score_

0.8044852191641182

In [89]:
gcv.best_params_

{'min_impurity_decrease': 0.0}

### geting best model

In [90]:
model_1=DecisionTreeClassifier()
model_2=DecisionTreeClassifier(min_samples_split=150)
model_3=DecisionTreeClassifier(min_samples_split=150,max_depth=30,min_samples_leaf= 20,max_features=10,max_leaf_nodes=20)


In [94]:
score=cross_val_score(model_1,x,y,cv=5,scoring='accuracy')
score.mean()

0.7177441077441078

In [97]:
score=cross_val_score(model_2,x,y,cv=5,scoring='accuracy')
score.mean()

0.8044682296975875

In [98]:
score=cross_val_score(model_3,x,y,cv=5,scoring='accuracy')
score.mean()

0.8044852191641182